In [1]:
import requests as rq
from bs4 import BeautifulSoup
import ujson
from tqdm.auto import tqdm

In [2]:
# Get style list
with open('saatchi style list.html') as f:
    html = f.read()

soup = BeautifulSoup(html)
styles_html = soup.findAll('a')
styles = [(x.get('href')).replace('/paintings/', '').replace('?hitsPerPage=100', '') for x in styles_html if x.get('href')]
styles

['abstract',
 'fine-art',
 'modern',
 'abstract-expressionism',
 'expressionism',
 'figurative',
 'impressionism',
 'realism',
 'conceptual',
 'portraiture',
 'surrealism',
 'pop-art',
 'minimalism',
 'illustration',
 'art-deco',
 'street-art',
 'photorealism',
 'folk',
 'cubism',
 'documentary',
 'dada']

In [16]:
base_url = 'https://www.saatchiart.com/paintings/'
json_content_list = []

for style in tqdm(styles[:4]):
    params = {'hitsPerPage': '100',
          'page': 1}
    url = base_url + style
    print(url)
    page_html = rq.get(url, params=params)
    soup = BeautifulSoup(page_html.content)
    last_page_number = int(soup.find('a', {'data-type': 'last-page'}).get('data-value'))

    with tqdm(total=last_page_number) as pbar:
        # while params['page'] <= 2:
        while params['page'] <= last_page_number:

            params.update({'page': params['page'] + 1})
            page_html = rq.get(url, params=params)
            soup = BeautifulSoup(page_html.content)
            page_json_dict = ujson.loads(soup.find('script', {'id': '__NEXT_DATA__'}).contents[0])
            json_content_list.append(page_json_dict['query']['page']['data']['algoliaResults']['hits']['results'])
            pbar.update(1)

  0%|          | 0/4 [00:00<?, ?it/s]

https://www.saatchiart.com/paintings/abstract


  0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
l = []


for i in json_content_list:
    l = l + i
len(l)

1400

In [12]:
page_html.url

'https://www.saatchiart.com/paintings/abstract-expressionism?hitsPerPage=100&page=3'

In [94]:
for style in styles:
    url = url + style
    print(url)
    page_html = rq.get(url, params=params)
    soup = BeautifulSoup(page_html.content)
    last_page_number = soup.find('a', {'data-type': 'last-page'}).get('data-value')
    # page_html_list.append(rq.get(url, params=params))

    while params['page'] <= last_page_number:
        params.update('page', params['page']+1)
        soup = BeautifulSoup(page_html.content)
        image_url_list = list(soup.findAll('img'))
        for url in image_url_list:
            if url.get('data-src') and url.get('data-src')[:2] != '//':
                urls.append(url.get('data-src'))

In [96]:
urls = [url.replace('-6.jpg', '-7.jpg') for url in urls]

In [5]:
params = {'hitsPerPage': '100',
          'page': 1}
url = 'https://www.saatchiart.com/paintings/'
page_html = rq.get(url, params=params)
soup = BeautifulSoup(page_html.content)
last_page_number = soup.find('a', {'data-type': 'last-page'}).get('data-value')
last_page_number

'250'

In [7]:

# soup.find('script', {'id': '__NEXT_DATA__'}).contents
j = ujson.loads(soup.find('script', {'id': '__NEXT_DATA__'}).contents[0])

x2 = j['query']['page']['data']['algoliaResults']['hits']['results']

b'<!DOCTYPE html><html lang="en"><head><meta http-equiv="x-ua-compatible" content="ie=edge"/><meta name="google-site-verification" content="D0TrMFxM-EpPQHRE3_NhEkyr18KTU2oAKWthx_UiPEw"/><link rel="dns-prefetch" href="//www.saatchiart.com"/><link rel="preconnect" href="https://cloudfront.net/" crossorigin="anonymous"/><link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous"/><link rel="preconnect" href="https://www.googletagmanager.com" crossorigin="anonymous"/><link rel="preconnect" href="https://api.visii.com/" crossorigin="anonymous"/><link rel="preload" href="https://d3t95n9c6zzriw.cloudfront.net/static/css/fonts.min.css" as="style"/><script>dataLayer = [{"environment":"production","event":"pagetype","pagetype":"browse","pagination":1,"sort":"default","subpagetype":"originals","category":"Paintings","color":"all","country":"all","material":"all","medium":"all","orientation":"all","price":"all","size":"all","style":"all","subject":"all"}]</script><script

In [18]:
l

[{'artId': 2638433,
  'artistArtworksUrl': 'https://www.saatchiart.com/account/artworks/420935',
  'artistRecognition': [],
  'artworkId': 2638433,
  'artworkRank': 322,
  'artworkTitle': 'Optical de la serie TRAMAS OPTiCAS',
  'artworkUrl': 'https://www.saatchiart.com/art/Painting-Optical-de-la-serie-TRAMAS-OPTiCAS/420935/2448313/view',
  'aspectRatio': 0.98302263648469,
  'category': 'Paintings',
  'city': 'BUENOS AIRES',
  'color': '#2C1378',
  'colorProximity': 39,
  'country': 'Argentina',
  'curatedStatus': 1,
  'depth': 2,
  'firstName': 'Adriana',
  'hasLimitedEditions': False,
  'hasOpenEditions': True,
  'height': 40,
  'idUser': 420935,
  'imageUrl': 'https://images.saatchiart.com/saatchi/420935/art/2638433/1708326-YEGGUYCZ-6.jpg',
  'isSafe': True,
  'isSeoValid': True,
  'keywords': ['pink',
   'repetition',
   'black',
   'shape',
   'op art',
   'OPTICO ',
   'abstract',
   'geometric',
   'minimal',
   'modern',
   'optical'],
  'lastName': 'Ablin Abstract Painter',
  '